In [ ]:
# ..... use new merged atlas ..... #
# mix of Heinz and M82, tissues/stages annotated by tissue categories, all non-normalized

In [3]:
## ..... functions to calculate coexpression and coexpression neighborhood conservation for all pairs of genes in the data ..... ##
build_coexp_network <- function(net, method = "spearman", flag = "rank"){
    
    # Calculate correlation coefficients
    genes = rownames(net)
    net = net[rowSums(net) > 0, ]

#     print(paste0("... ",dim(net)[1]," of ",length(genes), " genes have non-zero expression" ))

    net = cor(t(net), method = method)
    
    # Create network
    temp = net[upper.tri(net, diag = T)]
    if (flag == "abs"){
        temp = abs(temp)
    }
    
#     print("...ranking")
    temp = rank(temp, ties.method = "average")  

#     print("...reconstructing matrix")
    net[upper.tri(net, diag = T)] = temp

    net = t(net)
    net[upper.tri(net, diag = T)] = temp

    net = net / max(net, na.rm = T)
    med = median(net, na.rm = T)

    ind = setdiff(genes, rownames(net))   # which genes are missing?

    temp = matrix(med, length(ind), dim(net)[2])
    rownames(temp) = ind

    net = rbind(net, temp)
    temp = matrix(med, dim(net)[1], length(ind))
    colnames(temp) = ind
    net = cbind(net, temp)

    # reorder to original
    net = net[genes, genes]
    diag(net) = 1

    return(net)
}

get_top_pairs_mat <- function (mat.rank, n){ 
  f = mat.rank>dim(mat.rank)[1]-n
  mat.pairs = mat.rank*0
  mat.pairs[f] = 1
  return(mat.pairs)
}   

auroc_analytic3 <- function (mat, np, nL){          
  ranks = 1:nL
  dr = np*(nL - np)
  aa = lapply(1:dim(mat)[1], function(i) (mat[i,]-sum(ranks[1:np[i]]))/(dr[i]))                           
  auroc <- t(matrix(unlist(aa), nrow = nL))
  return(auroc)
}                

get_NM_labels <- function(omnew, matn){
  mat_labels = omnew
  for (j in 1:dim(matn)[1]){   
        if(length(which(matn[j,]==1))>1){   # many genes have just 1 top hit in meta nets - don't want to consider these
            mat_labels[j,] = colSums(omnew[which(matn[j,]==1),])  
        } 
  }
  return(mat_labels)
} 

# specificity calculation
calc_spec <- function (res, np, nL){    
  
  ranks = 1:nL
  mini = sum(ranks[1:np])
  range = np*(nL - np)
  
  temp = t(apply(res, 1, function(x) rank(x, ties.method = "average")))      
  spec <- (temp - mini)/range   
  #spec = spec*orthologmat
  
  return(spec)
}  

# aggregate specificity calculation
calc_agg_spec <- function (res, orthologmat, nL){    
  
  temp = t(apply(res, 1, function(x) rank(x, ties.method = "average")))  # row-wise ranks
  temp2 = rowSums(temp*orthologmat)
  ranks = 1:nL
  np = rowSums(orthologmat)
  dr = np*(nL - np)
  
  bb = lapply(1:length(temp2), function(i) (temp2[i]-sum(ranks[1:np[i]]))/(dr[i]))                           
  aggspec <- t(matrix(unlist(bb), nrow = 1))
  return(aggspec)
}

In [4]:
# load TPM-normalized 25-tissue expression atlas
tom3 = as.matrix(read.delim('tomato_expression_atlas_25_tissue.csv', sep = ','))
tom3[1000:1002,]

,TL5,evm_leaf_primordia,lvm_leaf_primordia,mvm_leaf_primordia,sym_leaf_primordia,tm_leaf_primordia,M82_apices,M82_pollen,Heintz_Root,Heinz_10dBreakerFruit,⋯,Heinz_FlowerBud,Heinz_Leaf,Heinz_MatureGreenFruit,M82_EVM,M82_FM,M82_LVM,M82_MVM,M82_SIM,M82_SYM,M82_TM
Solyc01g010215,7.146936,14.20113,52.26842,27.57112,136.69053,87.12926,75.98714,42.10089,53.35794,53.97858,⋯,14.605567,16.225133,15.524247,13.188670,0,0,0,0,0,0
Solyc01g010220,8.270879,11.33616,43.55239,19.26256,71.21647,54.85853,40.68079,16.22233,32.69143,35.64889,⋯,8.079807,11.401773,9.502044,21.270478,0,0,0,0,0,0
Solyc01g010230,24.204666,17.42701,244.40861,205.89054,171.46499,234.20875,167.78112,66.23232,80.18749,83.17275,⋯,1.079160,3.077971,1.865762,1.862704,0,0,0,0,0,0


In [11]:
# load same dataset with additional columns containing tissue-specificity, no. of tissues where gene has higher-than-median expression, expression breadth
tom3p = read.delim('tomato_gene_exp_atlas_25_tissue.csv', sep = ',')
tom3p[1000:1002,]

,TL5,evm_leaf_primordia,lvm_leaf_primordia,mvm_leaf_primordia,sym_leaf_primordia,tm_leaf_primordia,M82_apices,M82_pollen,Heintz_Root,Heinz_10dBreakerFruit,⋯,M82_FM,M82_LVM,M82_MVM,M82_SIM,M82_SYM,M82_TM,expr,num_tissues_exp,exp_type,tau
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<int>,<int>,<int>,<int>,<int>,<int>,<chr>,<int>,<chr>,<dbl>
Solyc01g010215,7.146936,14.20113,52.26842,27.57112,136.69053,87.12926,75.98714,42.10089,53.35794,53.97858,⋯,0,0,0,0,0,0,expressed,19,ubiquitous,0.8071723
Solyc01g010220,8.270879,11.33616,43.55239,19.26256,71.21647,54.85853,40.68079,16.22233,32.69143,35.64889,⋯,0,0,0,0,0,0,expressed,19,ubiquitous,0.7547915
Solyc01g010230,24.204666,17.42701,244.40861,205.89054,171.46499,234.20875,167.78112,66.23232,80.18749,83.17275,⋯,0,0,0,0,0,0,expressed,16,broad,0.7819432


In [9]:
# gene is expressed if it has at least median expression in the tissue
sum(tom3p$num_tissues_exp==0)
dim(tom3)
tom3 = tom3[which(tom3p$num_tissues_exp!=0),]
dim(tom3)

[1] 10167

[1] 34109    25

[1] 23942    25

In [12]:
# gene is expressed if it has at least median expression in the tissue
sum(rowSums(tom3p[,1:25])!=0)

[1] 29315

In [10]:
# get coexp network for Anat's new ITAG4.0 merged tomato data
prionet_atlas = build_coexp_network(tom3, method = 'pearson')

In [21]:
# save atlas coexpression data for 23k genes
prionet_atlas[1:2,1:2]

library(rhdf5)
rfile = 'tomato_atlas_25_tissue_coexpression.h5'   
               
suppressWarnings(h5createFile(rfile)) 
h5createDataset(rfile, "prionet_atlas", c(dim(prionet_atlas)[1],dim(prionet_atlas)[2]), chunk = c(101,101))
h5createDataset(rfile, "sp_netid", dims = dim(prionet_atlas)[1], storage.mode = "character", size = max(1+nchar(rownames(prionet_atlas))))
h5createDataset(rfile, "chunkSize", 1)

h5write(prionet_atlas, rfile, "prionet_atlas")         
h5write(rownames(prionet_atlas), rfile, "sp_netid")  
h5write(101, rfile, name = "chunkSize")      
h5closeAll()   

,Solyc00g500023,Solyc00g500062
Solyc00g500023,1.00000,0.99864
Solyc00g500062,0.99864,1.00000


In [ ]:
# .....     conservation      ..... #

In [11]:
# calculate FC/spec for atlas data - BBH top 10s
diag(prionet_atlas) = 0
rowrankmat = t(apply(prionet_atlas, 1, rank, ties.method = 'random'))

n = 10
mini1 = sum(1:n)      # sum of least 10 ranks; FC
range1 = n*(dim(prionet_atlas)[1] - n)
mini2 = 1       # least rank; spec
range2 = dim(prionet_atlas)[1] - 1

mat1 = get_top_pairs_mat(rowrankmat, n)
gg_sp12 = mat1%*%t(rowrankmat)    # sp1 genes predict sp2 genes this well
fc_atlas = (gg_sp12 - mini1)/range1
rownames(fc_atlas) = rownames(prionet_atlas)
colnames(fc_atlas) = rownames(prionet_atlas)

diag(gg_sp12) = 0
rank_ggsp12 = t(apply(gg_sp12, 1, rank, ties.method ='average'))   # rank for spec
sc_atlas = (rank_ggsp12 - mini2)/range2
rownames(sc_atlas) = rownames(prionet_atlas)
colnames(sc_atlas) = rownames(prionet_atlas)

sc_atlas[1:5,1:5]
dim(sc_atlas)
head(which(fc_atlas>1))

,Solyc00g500023,Solyc00g500062,Solyc00g500063,Solyc00g500065,Solyc00g500066
Solyc00g500023,0.0000000,0.9994152,0.9984128,0.9985798,0.9979742
Solyc00g500062,0.9995823,0.0000000,0.9985590,0.9982039,0.9968673
Solyc00g500063,0.9993735,0.9522159,0.0000000,0.9998329,0.9980368
Solyc00g500065,0.9991228,0.9994361,0.9541373,0.0000000,0.9984336
Solyc00g500066,0.9986634,0.9969717,0.9550979,0.9551397,0.0000000


[1] 23942 23942

integer(0)

In [23]:
# save files
rfile = 'tomato_atlas_25_tissue_coexp_cons.h5'  
               
suppressWarnings(h5createFile(rfile)) 
h5createDataset(rfile, "fc_atlas", c(dim(fc_atlas)[1],dim(fc_atlas)[2]), chunk = c(101,101))
h5createDataset(rfile, "sc_atlas", c(dim(fc_atlas)[1],dim(fc_atlas)[2]), chunk = c(101,101))
h5createDataset(rfile, "sp_netid", dims = dim(fc_atlas)[1], storage.mode = "character", size = max(1+nchar(rownames(fc_atlas))))
h5createDataset(rfile, "chunkSize", 1)

h5write(fc_atlas, rfile, "fc_atlas")
h5write(sc_atlas, rfile, "sc_atlas")          
h5write(rownames(fc_atlas), rfile, "sp_netid")  
h5write(101, rfile, name = "chunkSize")      
h5closeAll() 

In [ ]:
# .....     get scores for Sri's list of tomato paralogs      ..... #

In [12]:
genes = read.delim('tomato_paralogs_Sri.csv', sep = ',')
genes[1:3,]

,Gene1,Gene2,OGs
,<chr>,<chr>,<chr>
1,Solyc01g014055,Solyc01g056575,OG0000006
2,Solyc01g014055,Solyc01g058003,OG0000006
3,Solyc01g014055,Solyc01g066055,OG0000006


In [13]:
genes$exp1 = tom3p$num_tissues_exp[match(genes$Gene1, rownames(tom3p))]
genes$exp2 = tom3p$num_tissues_exp[match(genes$Gene2, rownames(tom3p))]
tail(genes)
length(which(genes$exp1>0& genes$exp2>0))

,Gene1,Gene2,OGs,exp1,exp2
,<chr>,<chr>,<chr>,<int>,<int>
25342,Solyc12g049611,Solyc12g049614,OG0036202,19,0
25343,Solyc12g049611,Solyc12g049616,OG0036202,19,9
25344,Solyc12g049614,Solyc12g049616,OG0036202,0,9
25345,Solyc01g103310,Solyc01g150153,OG0038467,0,0
25346,Solyc02g088950,Solyc02g088980,OG0038488,4,3
25347,Solyc08g044300,Solyc08g150123,OG0038616,0,0


[1] 4051

In [14]:
# get coexp and cons scores for tomato paralogs
genes$coexp_atlas = NA
genes$FC_atlas = NA
genes$spec_atlas = NA
pb = txtProgressBar(min = 0, max = dim(genes)[1], initial = 0)

for(ii in 1:dim(genes)[1]){
    id1 = match(unlist(genes[ii,1:2]), rownames(prionet_atlas))    
    
    if(!is.na(sum(id1))){
        genes$coexp_atlas[ii] <- prionet_atlas[id1[1], id1[2]]
        genes$FC_atlas[ii] <- 0.5*(fc_atlas[id1[1], id1[2]] + fc_atlas[id1[2], id1[1]])
        genes$spec_atlas[ii] <- 0.5*(sc_atlas[id1[1], id1[2]] + sc_atlas[id1[2], id1[1]])
    }
            
    setTxtProgressBar(pb, ii)   
}

In [15]:
tail(genes)
dim(genes)
sum(!is.na(genes$coexp_atlas))

,Gene1,Gene2,OGs,exp1,exp2,coexp_atlas,FC_atlas,spec_atlas
,<chr>,<chr>,<chr>,<int>,<int>,<dbl>,<dbl>,<dbl>
25342,Solyc12g049611,Solyc12g049614,OG0036202,19,0,NA,NA,NA
25343,Solyc12g049611,Solyc12g049616,OG0036202,19,9,0.8225402,0.6773671,0.5723863
25344,Solyc12g049614,Solyc12g049616,OG0036202,0,9,NA,NA,NA
25345,Solyc01g103310,Solyc01g150153,OG0038467,0,0,NA,NA,NA
25346,Solyc02g088950,Solyc02g088980,OG0038488,4,3,0.7492072,0.7698583,0.8537446
25347,Solyc08g044300,Solyc08g150123,OG0038616,0,0,NA,NA,NA


[1] 25347     8

[1] 4051

In [89]:
tom3p['Solyc08g150123',]

,TL5,evm_leaf_primordia,lvm_leaf_primordia,mvm_leaf_primordia,sym_leaf_primordia,tm_leaf_primordia,M82_apices,M82_pollen,Heintz_Root,Heinz_10dBreakerFruit,⋯,M82_FM,M82_LVM,M82_MVM,M82_SIM,M82_SYM,M82_TM,expr,num_tissues_exp,exp_type,tau
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<int>,<int>,<int>,<int>,<int>,<int>,<chr>,<int>,<chr>,<dbl>
Solyc08g150123,0,0,0,0,0,1,2,0.5,0,0,⋯,0,0,0,0,0,0,low_exp,0,specific,0.9583333


In [16]:
# save file
write.table(genes[,-c(4:5)], file = 'tomato_paralogs_atlas_25_tissue_Sri_scores.csv', sep = ',', row.names = F, col.names = T)